## Quick Scrum
Quick Scrum
qs take <story>
qs store <store>
qs init 
qs touch <story>


### Initialize a new project

>_qs init [project_name]_

Initialize a new Quick Scrum project in current folder. Current folder name will be used as project name when the name is not provided.
The command create a folder .qs where stores (backlog and sprints) are created. If the current folder or its parent is a Git repository, the .qs folder is automatically added to the Git repository.

Current user is added as manager of the project. Store is set to backlog.

### Select a store

>_qs store [bl|sb|1|2|3...]_

Set the current store in the project. Quick-scrum looks for a project in current folder or parent folders.
The possible stores are:
    - bl: backlog
    - sb: sandbox
    - 1: sprint #1
    - 2: sprint #2
and so on.

When called without a target store, it shows the list of stores and the one that is current. It allows than to choose the store to use.

    





### List stories

>_qs top [n]_

Show the top _n_ stories in the current store or all stories when _n_ is not provided

### New story

>_qs new [story_id]_

Create a new story with the specified id

### Touch a story

>_qs touch [story] [pos]_

Touch the story, i.e. makes it top of the list or at a specific position in the list

### Edit a story

>_qs edit [story]_

Edit a story content. the parameter *story* can be either the story name or the progressive number. When no parameter is provided, it shows the available stories in the current store


In [11]:
#export
from ascrum.core.project import Project
from ascrum.core.store import Store
from ascrum.core.config import config
import getpass
import time
from pathlib import Path
import os
import sys
import subprocess

In [15]:
# default_exp main
#export
import argparse

def get_init_parser(subparsers):
    parser = subparsers.add_parser('init', help='create a Quick Scrum project')
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where to init the project')

def get_store_parser(subparsers):
    parser = subparsers.add_parser('store', help='select the current store')
    parser.add_argument('name',
                        nargs='?',
                        default='',
                        help='store to use; sb for sandbox, bl for backlog, 1 for sprint-1...')
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where the project is')

def get_top_parser(subparsers):
    parser = subparsers.add_parser('top', help='shows the top stories in the current store')
    parser.add_argument('n',
                        nargs='?',
                        default=10,
                        help='number of stories to show')
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where the project is')

def get_edit_parser(subparsers):
    parser = subparsers.add_parser('edit', help='edit a story in the current store')
    parser.add_argument('story',
                        nargs='?',
                        help='story to edit or partial name of it',
                        default=None)
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where the project is')

def get_touch_parser(subparsers):
    parser = subparsers.add_parser('touch', help='touch a story, making it the highet priority in the current store')
    parser.add_argument('story',
                        nargs='?',
                        help='story to edit or partial name of it',
                        default=None)
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where the project is')

def get_new_parser(subparsers):
    parser = subparsers.add_parser('new', help='create a new story in current store')
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where the project is')

def get_web_parser(subparsers):
    parser = subparsers.add_parser('web', help='start the web ui')
    parser.add_argument('--port', '-p',
                        help='port for the web server',
                        default=4444)
    parser.add_argument('--debug', '-d',
                        help='set if to start in debug mode',
                        default=False)
def get_pwd_parser(subparsers):
    parser = subparsers.add_parser('pwd', help='set the password for web access. Not required if the user has pam access')
    parser.add_argument('user',
                        help='username',
                        default=None)

def get_users_parser(subparsers):
    parser = subparsers.add_parser('users', help='manage users in the project')
    parser.add_argument('--path', '-p',
                        default='.',
                        help='path where the project is')

    subparsers2 = parser.add_subparsers(dest='users_command')
    parser = subparsers2.add_parser('add', help='add a new user to the project')
    parser.add_argument('user',
                        help='the username')

    parser = subparsers2.add_parser('edit', help='edit user information')    
    parser.add_argument('user',
                        help='the username')

    parser = subparsers2.add_parser('remove', help='remove a user from the project')    
    parser.add_argument('user',
                        help='the username')

    parser = subparsers2.add_parser('ls', help='list users in current project')


def get_arg_parser():
    parser = argparse.ArgumentParser(prog='qs', description='Quick Scrum 1.0')
    subparsers = parser.add_subparsers(dest='command')

    get_init_parser(subparsers)
    get_store_parser(subparsers)
    get_top_parser(subparsers)
    get_new_parser(subparsers)
    get_edit_parser(subparsers)
    get_touch_parser(subparsers)
    get_pwd_parser(subparsers)
    get_users_parser(subparsers)
    get_web_parser(subparsers)
    return parser

In [13]:
#export    
def choose_option(options, title):
    print(title)
    while True:
        [print(f'[{1+i}] {option}') for i,option in enumerate(options)]
        choice = input(title)
        if not choice: return None

        if choice.isnumeric():
            choice = int(choice)
            if choice-1 in range(0, len(options)):
                return options[choice-1]

def input_until(title, condition=lambda s: not len(s), msg='Invalid input. Please try again!'):
    value = input(title)
    while condition(value):
        print(f'\033[31m{msg}\033[39m')
        value = input(title)
    return value


def print_context(p,s):
    title = f'{p.name}/{s.name}'
    print(f'\033[33m{title}')
    print('-'*len(title)+'\033[39m')

def print_users(users):
    print(f'\033[33m{title}')

In [14]:
#export
def assert_project(p):
    if p == None:
        cf = Path('.').resolve()
        print(f'No quick-scrum project in current folder {cf}. Use command init to initialize a new project')
        sys.exit()

def get_current_store(p):
    try:
        cs = (p.path/p.project_folder/'current-store').resolve(True)
        return p.get_store(cs.stem)
    except FileNotFoundError:
        return None

def set_current_store(p, store_name):
    s = p.get_store(store_name)
    (p.path/p.project_folder/'current-store').symlink_to(s.path)
    return s

def get_context(path):
    p = Project.find(Path(path))
    assert_project(p)
    s = get_current_store(p)
    return p, s if s else set_current_store(p, 'backlog')

def start_editor(fpath):
    if not config.editor:
        print(f'Define an editor in qs.yaml')
        sys.exit(1)
    cmd = f'{config.editor} "{fpath.resolve().as_posix()}"'
    subprocess.call(cmd, shell=True)

In [5]:
#export
def init_command(path):
    p = Project(Path(path).absolute())
    if p.exists():
        raise AttributeError(f'A project already exists in {p.path}')

    description = input('Project description: ')
    project = {
        'description': description,
        'creator': getpass.getuser()
    }
    p.init(project)
    print(f'A Scrum project created in  {p.path}')


def store_command(name, path):
    p, s = get_context(path)
    print_context(p,s)

    stores = p.list_stores()
    if not name or not name in stores:
        name = choose_option(stores,  'Choose the store: ')
        if name:
            s = p.get_store(name)
            (p.path/p.project_folder/'current-store').unlink()
            (p.path/p.project_folder/'current-store').symlink_to(s.path, target_is_directory=True)


def top_command(n, path):
    p, s = get_context(path)
    print_context(p,s)

    paths = sorted(s.path.glob('*.yaml'), key=os.path.getmtime, reverse=True)
    if not n: paths = paths[0:n]
    [print(f.name) for f in paths]


def select_story(s, pattern, msg):
    stories = s.keys()
    if pattern:
        stories = [story for story in s.keys() if pattern in story]

    if not stories:
        return None
    elif len(stories) == 1:
        return stories[0]
    else:
        return choose_option(stories, msg)

def new_command(path):
    p, s = get_context(path)
    print_context(p,s)

    print('Create a new story')
    story_summary = input('Summary: ', )
    story_id = p.new_story_id()

    users = p.list_users()
    story_owner = choose_option(users, 'Choose the owner: ')

    key = f'{story_id}. {story_summary}'
    s[key] = {
        'description': 'FILL ME',
        'points': 1,
        'state': 'TODO',
        'owner': story_owner,
        'tasks': []
    }
    start_editor(s.get_path(key))
    print(f'Story "{key}" created in {p.name}/{s.name}')

def edit_command(story_name, path):
    p, s = get_context(path)
    print_context(p,s)

    story_name = select_story(s, story_name, "Choose the story to edit: ")
    if story_name:
        fpath = s.get_path(story_name)
        start_editor(fpath)

def touch_command(story_name, path):
    p, s = get_context(path)
    print_context(p,s)

    story_name = select_story(s, story_name, "Choose the story to touch: ")
    if story_name:
        s.touch(story_name)
        path = s.get_path(story_name)
        mtime = path.stat().st_mtime
        print(f'Touch {story_name} at {mtime}')

def web_command(port, debug):
    from ascrum.server import start_server
    import ascrum.api.project
    import ascrum.api.user
    import ascrum.api.library
    import ascrum.api.store
    import webbrowser
    start_server(port, debug)
    #webbrowser.open('http://www.python.org')

def pwd_command(user):
    import getpass
    pwd = getpass.getpass(f"Enter a password for user {user} web access: ")
    config.set_password(user, pwd)

def users_command(command, user, path):
    p, s = get_context(path)
    print_context(p,s)

    if command == 'add':
        fpath = p.set_user(user, {
            'user': user,
            'email': '',
            'picture': '',
        })
        start_editor(fpath)
    elif command == 'edit':
        fpath = p.get_user_path(user)
        start_editor(fpath) if fpath else print(f'Unknown user {user}') 
    elif command == 'remove':
        fpath = p.get_user_path(user)
        if fpath:
            op = choose_option(['Yes','No'], f'Do you really want to remove user {user}?')
            if op == 'Yes':
                p.set_user(user, None)
        else:
            print(f'Unknown user {user}') 
    elif command == 'ls':
        print(','.join(p.list_users()))


In [1]:
#export
def dispatch_command(args=None):
    p = get_arg_parser()
    args = p.parse_args(args)

    if args.command == 'init':
        init_command(args.path)
    elif args.command == 'store':
        store_command(args.name, args.path)
    elif args.command == 'top':
        top_command(args.n, args.path)
    elif args.command == 'new':
        new_command(args.path)
    elif args.command == 'edit':
        edit_command(args.story, args.path)
    elif args.command == 'touch':
        touch_command(args.story, args.path)
    elif args.command == 'web':
        web_command(args.port, args.debug)
    elif args.command == 'pwd':
        pwd_command(args.user)
    elif args.command == 'users':
        print(args)
        users_command(args.users_command, args.user, args.path)

In [8]:
dispatch_command(['users', '-p /tmp/myscrum/', 'add', 'hei'])

No quick-scrum project in current folder /home/mp/Github/ascrum/nbs. Use command init to initialize a new project


SystemExit: 

In [3]:
#export
if __name__ == '__main__':
    try:
        dispatch_command()
    except Exception as e:
        print('Error:', e)

__main__
name 'dispatch_command' is not defined


In [ ]:
p.parse_args??

